In [8]:
import json
import numpy as np
import pandas as pd

In [147]:
isDebug = False

In [111]:
def order_json_df(orders):
    if isDebug:
       print("orders in order_json_df:")
       print(orders)
    df = []
    for type_of_recipe in orders:
        sub_recipes = orders[type_of_recipe]
        if isDebug:
           print("sub_recipes in order_json_df:")
           print(sub_recipes)
        for i in range(len(sub_recipes)):
            size_recipes = sub_recipes[i]
            #
            if isDebug:
               print("size_recipes:")
               print(size_recipes)
            number_of_recipes = size_recipes["recipes"]
            for number in ['two_portions', 'four_portions']: 
                if number in size_recipes:
                   if number == 'two_portions':
                      count = 2
                   else:
                      count = 4
                   elem = {"number_of_customers": size_recipes[number], 
                           "number_of_portions": count,
                           "number_of_recipes": number_of_recipes, 
                           "type_of_recipes": type_of_recipe} 
                   df.append(elem)
    return pd.DataFrame(df)

In [112]:
def recipes_mapping_AD(stock_amount, number_of_customers, number_of_portions, number_of_recipes):
    """
    recipes_mapping is mapping between customer order and stock
    -----------------------------------------------------------
    Inputs:
    stock_amount is numpy array of the number of recipes in the order given the diet type;
    number_of_customers is Int number, the number of customers in the order
                           for given number of portions, number of recipes and diet type;
    number_of_portions is Int number, the number of portions for given number of recipes  and diet type;  
    number_of_recipes  is Int number, the number of recipes  for given number of portions and diet type;
    ----------------------------------------------------------------------------------------------------
    Outputs:
    isMappingGood is True if we could map successfully the order to stock and False otherwise
    remained_stock_amount is the stock that left after mapping the order
    not_satisfied_number_of_customers i the number of customers that we cannot fulfill the order
    """
    if isDebug:
       print("Start of recipes_mapping_AD")
       print("stock_amount, number_of_customers, number_of_portions, number_of_recipes")
       print(stock_amount, number_of_customers, number_of_portions, number_of_recipes)
    #
    isMappingGood = True
    #
    #Check if the number of recipes and its amount satisfy costrains
    if (sum(stock_amount) < number_of_recipes * number_of_portions) or (len(stock_amount) < number_of_recipes):
       isMappingGood = False 
       remained_stock_amount = stock_amount
    else:
       #
       # Algorithm:
       # processed_number_of_customers = 0
       # while processed_number_of_customers < number_of_customers perform the following:
       # 1) Select number_of_recipes recipes in stock with number_of_recipes largest stock amount available;
       # 2) Find the number of customers that we can fulfil the order given the number of portions and recipes obtained in Step 1, min_number_of_customers;
       # 3) If min_number_of_customers == 0 then we  cannot fulfil order and EXIT
       #                                    else GOTO 4)
       # 4) Subtract min_number_of_customers * number_of_portions from the stock for recipes obtained in Step 1.
       # 5) processed_number_of_customers += min_number_of_customers.
       #
       processed_number_of_customers = 0 
       while processed_number_of_customers < number_of_customers:
           # Get the recipes with the most stock remaining 
           recipes_position_w_largest_stock = np.argpartition(stock_amount, -number_of_recipes)[-number_of_recipes:]
           if isDebug:
              print("Before stock_amount change")
              print("stock_amount:")
              print( stock_amount)
              print("np.sort(stock_amount):")
              print( np.sort(stock_amount))
              print("np.sort(stock_amount)[-number_of_recipes]")
              print( np.sort(stock_amount)[-number_of_recipes])
              print("recipes_position_w_largest_stock:") 
              print( recipes_position_w_largest_stock)
           #
           # find the number of customers that we can fullfil the order given the number of portions    
           min_number_of_customers = min(stock_amount[recipes_position_w_largest_stock] // number_of_portions)
           # 
           min_number_of_customers = min([min_number_of_customers, number_of_customers - processed_number_of_customers]) 
           #
           if isDebug:
              print("min_number_of_customers in recipes_mapping_AD: ")
              print( min_number_of_customers)
           #
           if min_number_of_customers != 0:
              stock_amount[recipes_position_w_largest_stock] = \
                    stock_amount[recipes_position_w_largest_stock] - min_number_of_customers * number_of_portions  
           else:     
              isMappingGood = False
              break
           #
           if isDebug:
              print("After stock_amount change")
              print("stock_amount:")
              print( stock_amount)
           #
           if isDebug:
              print("Before update processed_number_of_customers: ", processed_number_of_customers)
              print("min_number_of_customers:", min_number_of_customers)
           #
           processed_number_of_customers += min_number_of_customers
           if isDebug:
              print("After update processed_number_of_customers: ", processed_number_of_customers)
              print("number_of_customers:", number_of_customers)
       #
       remained_stock_amount = stock_amount
       #
    return isMappingGood, remained_stock_amount

In [149]:
def isMatchingOrder(stock, orders):
    """
    Determine whether stocked recipes can satisfy orders.

    Parameters
    ----------
    stock : stock dict, loaded from json.
    example of stock:
               {'recipe_1': {'stock_amount': 8, 'type_of_recipes': 'vegetarian'},
                'recipe_2': {'stock_amount': 8, 'type_of_recipes': 'vegetarian'},
                'recipe_3': {'stock_amount': 8, 'type_of_recipes': 'vegetarian'},
                'recipe_4': {'stock_amount': 8, 'type_of_recipes': 'vegetarian'},
                'recipe_5': {'stock_amount': 8, 'type_of_recipes': 'gourmet'   }
               }
 
    orders : orders dict, loaded from json.
    example of order:
               {'vegetarian': [{"recipes": 2, 'two_portions': 1, 'four_portions': 11},
                               {"recipes": 3, 'two_portions': 2, 'four_portions': 9},
                               {"recipes": 4, 'two_portions': 5, 'four_portions': 6}
                              ],
                'gourmet':    [{"recipes": 2, 'two_portions': 3, 'four_portions': 21},
                               {"recipes": 3, 'two_portions': 4, 'four_portions': 1},
                               {"recipes": 4, 'two_portions': 5, 'four_portions': 2}
                              ]
               }
    # for example, {"recipes": 2, 'two_portions': 1, 'four_portions': 11} means
    # 1 customer will get two recipes 
    Returns
    ----------
    bool
        True if stock can satisfy the orders, False if not.
    """

    # Convert data to dataframes
    stock_df = pd.DataFrame(stock).T
    stock_df["recipe"] = list(stock_df.index)
    if isDebug:
       print("\n stock_df:")
       print(stock_df)
    # example of dataframe stock_df given the example of stock given above:
    #
    #          stock_amount  type_of_recipes     recipe
    # recipe_1     8          vegetarian       recipe_1
    # recipe_2     8          vegetarian       recipe_2
    # recipe_3     8          vegetarian       recipe_3
    # recipe_4     8          vegetarian       recipe_4
    # recipe_5     8          gourmet          recipe_5
    #
    orders_df = order_json_df(orders)
    if isDebug:
       print("\n orders_df:")
       print(orders_df)
    # example of dataframe order_df given the example of stock given above:
    #
    #   number_of_customers  number_of_portions  number_of_recipes  type_of_recipes
    # 0        1                    2                   2              vegetarian
    # 1        11                   4                   2              vegetarian
    # 2        2                    2                   3              vegetarian
    # 3        9                    4                   3              vegetarian
    # 4        5                    2                   4              vegetarian
    # 5        6                    4                   4              vegetarian
    # 6        3                    2                   2              gourmet
    # 7        21                   4                   2              gourmet
    # 8        4                    2                   3              gourmet
    # 9        1                    4                   3              gourmet
    # 10       5                    2                   4              gourmet
    # 11       2                    4                   4              gourmet
    #
    # Check whether there is enough stock of any kind
    stock_total = stock_df["stock_amount"].sum()
    #
    orders_total = orders_df["number_of_customers"] * orders_df["number_of_portions"] * orders_df["number_of_recipes"]
    orders_total = orders_total.sum()
    #
    # If demand is greater than stock, then demand cannot be satisfied
    if orders_total > stock_total:
        return False

    stock_amount = stock_df["stock_amount"].values.copy()
    veg_mask = (stock_df["type_of_recipes"] == "vegetarian").values  # Mask for vegetarian recipes

    # Iterate through subsets by most portions, then vegetarian, then most recipes
    orders_df.sort_values(["type_of_recipes", "number_of_portions", "number_of_recipes"], ascending = False, inplace=True)
    if isDebug:
       print("\n sorted orders_df:")
       print(orders_df)
    #
    # Assign orders for each customer subset

    for index, order in orders_df.iterrows():
        #
        if order["type_of_recipes"] == "vegetarian":
            if isDebug:
               print("Check Vegeterian!!!") 
               print("order:")
               print( order)
            # Vegetarian orders only
            veg_stock_amount = stock_amount[veg_mask].copy()
            isMappingGood, updated_veg_stock_amount = recipes_mapping_AD(veg_stock_amount, 
                                                                         order["number_of_customers"],
                                                                         order["number_of_portions" ], 
                                                                         order["number_of_recipes"  ])
            if not isMappingGood:
                # Orders cannot be satisfied
                break
            # Update stock
            stock_amount[veg_mask] = updated_veg_stock_amount
        else:
            if isDebug:
               print("Check gourmet!!!")
               print("order:")
               print( order)
            # Try gourmet recipes only
            gourmet_stock_amount = stock_amount.copy()
            isMappingGood, updated_gourmet_stock_amount = recipes_mapping_AD(gourmet_stock_amount,
                                                                             order["number_of_customers"],
                                                                             order["number_of_portions" ],  
                                                                             order["number_of_recipes"  ])
            # Update stock
            stock_amount = updated_gourmet_stock_amount

            if not isMappingGood:
               # Orders cannot be satisfied
               break
               # Update stock
    #
    return isMappingGood

In [150]:
with open('orders.json', "r") as json_file:
     orders = json.load(json_file)
#
orders

{'vegetarian': [{'recipes': 2, 'two_portions': 2, 'four_portions': 1},
  {'recipes': 3, 'two_portions': 3},
  {'recipes': 4, 'two_portions': 1, 'four_portions': 1}],
 'gourmet': [{'recipes': 2, 'two_portions': 1},
  {'recipes': 3, 'four_portions': 2}]}

In [151]:
orders = {'vegetarian': [{'recipes': 2, 'two_portions': 1, 'four_portions': 1},
  {'recipes': 3, 'two_portions': 3},
  {'recipes': 4, 'two_portions': 1, 'four_portions': 1}],
 'gourmet': [{'recipes': 2, 'two_portions': 1},
  {'recipes': 3, 'four_portions': 2}]}

In [152]:
with open('stock.json', "r") as json_file:
     stock = json.load(json_file)
#
stock

{'recipe_1': {'stock_amount': 8, 'type_of_recipes': 'vegetarian'},
 'recipe_2': {'stock_amount': 8, 'type_of_recipes': 'vegetarian'},
 'recipe_3': {'stock_amount': 14, 'type_of_recipes': 'vegetarian'},
 'recipe_4': {'stock_amount': 16, 'type_of_recipes': 'vegetarian'},
 'recipe_5': {'stock_amount': 4, 'type_of_recipes': 'vegetarian'},
 'recipe_6': {'stock_amount': 8, 'type_of_recipes': 'vegetarian'},
 'recipe_7': {'stock_amount': 8, 'type_of_recipes': 'gourmet'},
 'recipe_8': {'stock_amount': 6, 'type_of_recipes': 'gourmet'},
 'recipe_9': {'stock_amount': 6, 'type_of_recipes': 'gourmet'},
 'recipe_10': {'stock_amount': 8, 'type_of_recipes': 'gourmet'}}

In [153]:
stock = {'recipe_1': {'stock_amount': 8, 'type_of_recipes': 'vegetarian'},
 'recipe_2': {'stock_amount': 8, 'type_of_recipes': 'vegetarian'},
 'recipe_3': {'stock_amount': 14, 'type_of_recipes': 'vegetarian'},
 'recipe_4': {'stock_amount': 16, 'type_of_recipes': 'vegetarian'},
 'recipe_5': {'stock_amount': 4, 'type_of_recipes': 'vegetarian'},
 'recipe_6': {'stock_amount': 8, 'type_of_recipes': 'vegetarian'},
 'recipe_7': {'stock_amount': 8, 'type_of_recipes': 'gourmet'},
 'recipe_8': {'stock_amount': 6, 'type_of_recipes': 'gourmet'},
 'recipe_9': {'stock_amount': 6, 'type_of_recipes': 'gourmet'},
 'recipe_10': {'stock_amount': 8, 'type_of_recipes': 'gourmet'}}

In [154]:
orders_df = order_json_df(orders)
orders_df

,number_of_customers,number_of_portions,number_of_recipes,type_of_recipes
0,1,2,2,vegetarian
1,1,4,2,vegetarian
2,3,2,3,vegetarian
3,1,2,4,vegetarian
4,1,4,4,vegetarian
5,1,2,2,gourmet
6,2,4,3,gourmet


In [155]:
orders_df.sort_values(["type_of_recipes", "number_of_portions", "number_of_recipes"], ascending = False, inplace=True)
orders_df

,number_of_customers,number_of_portions,number_of_recipes,type_of_recipes
4,1,4,4,vegetarian
1,1,4,2,vegetarian
3,1,2,4,vegetarian
2,3,2,3,vegetarian
0,1,2,2,vegetarian
6,2,4,3,gourmet
5,1,2,2,gourmet


In [156]:
stock_df = pd.DataFrame(stock).T
stock_df["recipe"] = list(stock_df.index)
stock_df

,stock_amount,type_of_recipes,recipe
recipe_1,8,vegetarian,recipe_1
recipe_2,8,vegetarian,recipe_2
recipe_3,14,vegetarian,recipe_3
recipe_4,16,vegetarian,recipe_4
recipe_5,4,vegetarian,recipe_5
recipe_6,8,vegetarian,recipe_6
recipe_7,8,gourmet,recipe_7
recipe_8,6,gourmet,recipe_8
recipe_9,6,gourmet,recipe_9
recipe_10,8,gourmet,recipe_10


In [157]:
isMatchingOrder(stock, orders)

True